<a href="https://colab.research.google.com/github/ansonmiu0214/C490CW/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework: BERT

In [1]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 4.9MB/s 


In [1]:
# Imports
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import tqdm

from pytorch_pretrained_bert import BertConfig, BertTokenizer, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam

import matplotlib.pyplot as plt
%matplotlib inline

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'DEVICE={device}')

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=device))

DEVICE=cuda
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  | 

In [2]:
# Google Drive authorisation
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive

def in_gdrive(path):
    return os.path.join('/content/gdrive/My Drive/Colab Notebooks', path)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


ValueError: ignored

## Importing Data

In [0]:
import os

if not os.path.exists('enzh_data.zip'):
    !wget -O enzh_data.zip https://competitions.codalab.org/my/datasets/download/03e23bd7-8084-4542-997b-6a1ca6dd8a5f
    !unzip enzh_data.zip

TRAIN_EN = 'train.enzh.src'
TRAIN_ZH = 'train.enzh.mt'
TRAIN_SCORES = 'train.enzh.scores'
VAL_EN = 'dev.enzh.src'
VAL_ZH = 'dev.enzh.mt'
VAL_SCORES = 'dev.enzh.scores'
TEST_EN = 'test.enzh.src'
TEST_ZH = 'test.enzh.mt'

In [0]:
# Read from file

with open(TRAIN_EN) as f:
    train_en = f.readlines()
with open(TRAIN_ZH) as f:
    train_zh = f.readlines()
with open(TRAIN_SCORES) as f:
    train_scores = [float(score.strip()) for score in f]
with open(VAL_EN) as f:
    val_en = f.readlines()
with open(VAL_ZH) as f:
    val_zh = f.readlines()
with open(VAL_SCORES) as f:
    val_scores = [float(score.strip()) for score in f]
with open(TEST_EN) as f:
    test_en = f.readlines()
with open(TEST_ZH) as f:
    test_zh = f.readlines()

## Preprocessing

In [5]:
BERT_PRETRAINED_MODEL = 'bert-base-multilingual-cased'

# Load tokenizer
print(f'Loading tokenizer <{BERT_PRETRAINED_MODEL}>...', end=' ')
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL,
                                          do_lower_case=False)
print('done!')
print()

# Check tokenizer
sample_sent_id = 42

print('English')
print(train_en[sample_sent_id])
print(tokenizer.tokenize(train_en[sample_sent_id]))
print()

print('Chinese')
print(train_zh[sample_sent_id])
print(tokenizer.tokenize(train_zh[sample_sent_id]))

Loading tokenizer <bert-base-multilingual-cased>... done!

English
All 6 of the artillerymen recorded as wounded died).

['All', '6', 'of', 'the', 'artillery', '##men', 'recorded', 'as', 'wounded', 'died', ')', '.']

Chinese
据记录 ， 所有 6 名炮兵都受伤了) 。

['据', '记', '录', '，', '所', '有', '6', '名', '炮', '兵', '都', '受', '伤', '了', ')', '。']


In [0]:
def sentence_pairs_to_bert_input(tokenizer, en_sents, zh_sents, *, max_seq_length=256):

    bert_inputs = []

    for en_sent, zh_sent in zip(en_sents, zh_sents):

        en_tokens = tokenizer.tokenize(en_sent)
        zh_tokens = tokenizer.tokenize(zh_sent)

        num_bert_markers = 3

        total_length = len(en_tokens) + len(zh_tokens) + num_bert_markers
        if total_length > max_seq_length:
            raise Exception(f'Too long ({total_length})')
        
        tokens = ['[CLS]'] + en_tokens + ['[SEP]'] + zh_tokens + ['[SEP]']

        ids = tokenizer.convert_tokens_to_ids(tokens)

        padding = [0] * (max_seq_length - len(ids))

        ids_tensor = torch.LongTensor(ids + padding)

        bert_inputs.append(ids_tensor)

    return torch.stack(bert_inputs)

In [0]:
train_inputs = sentence_pairs_to_bert_input(tokenizer, train_en, train_zh, max_seq_length=132)
val_inputs = sentence_pairs_to_bert_input(tokenizer, val_en, val_zh, max_seq_length=132)
test_inputs = sentence_pairs_to_bert_input(tokenizer, test_en, test_zh, max_seq_length=132)

In [0]:
import math

def get_mini_batches(inputs, scores, *, batch_size=8):

    num_batches = math.ceil(len(inputs) / batch_size)

    for batch_id in range(num_batches):
        start_id = batch_id * batch_size
        end_id = (batch_id + 1) * batch_size
        yield inputs[start_id:end_id], scores[start_id:end_id]

## Model

In [9]:
# Utilities

from scipy.stats.stats import pearsonr

def RMSELoss(pred, target):
    return torch.sqrt(torch.mean((pred - target) ** 2))

print(torch.cuda.memory_summary(device=device))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [0]:
config = BertConfig(vocab_size_or_config_json_file=30522)

class BertForQE(nn.Module):
    def __init__(self):
        super().__init__()

        self.bert = BertForSequenceClassification.from_pretrained(BERT_PRETRAINED_MODEL, num_labels=1)
        # self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # self.out = nn.Linear(config.hidden_size, 1)

    def forward(self, input_ids):
        score = self.bert(input_ids)
        # pooled = self.dropout(pooled_output)
        # score = self.out(pooled_output)
        return score

In [11]:
model = BertForQE()
print(model)

LR = 0.003
optimiser = torch.optim.Adam(model.parameters(), lr=LR)

# optimiser = BertAdam(model.parameters(), lr=LR, schedule='warmup_linear', warmup=0.1, t_total=1000)

loss_fn = RMSELoss

BertForQE(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerN

In [0]:
def gradient_descent(model, loss_fn, optimiser, input_ids, scores):
    model.train()

    model.zero_grad()

    predictions = model(input_ids.to(device)).squeeze()
    
    loss = loss_fn(predictions, torch.Tensor(scores).to(device))

    loss.backward()

    optimiser.step()

    return loss

def train(model, loss_fn, optimiser, train_inputs, train_scores, val_inputs, val_scores, *, num_epochs=10, batch_size=32):

    for epoch_idx in range(num_epochs):
        print(f'Epoch #{epoch_idx + 1}')

        model.to(device)
        
        processed = 0
        increment = 0.05
        milestone = 0.05
        print('Training', end='')
        for input_ids, scores in get_mini_batches(train_inputs, train_scores, batch_size=batch_size):
            loss = gradient_descent(model, loss_fn, optimiser, input_ids, scores)
            processed += len(input_ids)

            if processed / len(train_inputs) >= milestone:
                print('.', end='')
                milestone += increment
        print('done!')
        print('Recent loss', loss)

        # Check validation loss
        model.eval()
        model.to('cpu')

        print('Getting validation predictions...', end='')
        val_preds = model(val_inputs[:50]).squeeze().detach()
        print(f'done! {val_preds[0]}..{val_preds[-1]}')
        val_loss = loss_fn(val_preds, torch.Tensor(val_scores[:50]))
        print('Validation loss', val_loss)

        pearson, _ = pearsonr(val_preds.numpy(), np.array(val_scores[:50]))
        print('Validation Pearson', pearson)

In [13]:
train(model, loss_fn, optimiser, train_inputs[:100], train_scores[:100], val_inputs, val_scores, batch_size=8)

Epoch #1
Training.............done!
Recent loss tensor(0.8049, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! -0.7593356966972351..-0.7593356966972351
Validation loss tensor(1.1567)
Validation Pearson 0.020010134978328507
Epoch #2
Training.............done!
Recent loss tensor(1.2965, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.27154621481895447..0.27154621481895447
Validation loss tensor(0.7492)
Validation Pearson -0.15317945884856743
Epoch #3
Training.............done!
Recent loss tensor(1.0521, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.06314269453287125..0.06314270198345184
Validation loss tensor(0.7395)
Validation Pearson 0.0981606542187859
Epoch #4
Training.............done!
Recent loss tensor(1.3717, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.17315824329853058..0.17315824329853058
Validation loss tensor(0.7373)
Validation Pearson nan

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Training.............done!
Recent loss tensor(1.1355, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! -0.018087113276124..-0.018087105825543404
Validation loss tensor(0.7515)
Validation Pearson 0.11258339568929353
Epoch #6
Training.............done!
Recent loss tensor(1.1619, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.05061214044690132..0.05061214044690132
Validation loss tensor(0.7408)
Validation Pearson nan
Epoch #7
Training.............done!
Recent loss tensor(1.2263, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.14069204032421112..0.14069204032421112
Validation loss tensor(0.7363)
Validation Pearson -0.08381997168645508
Epoch #8
Training.............done!
Recent loss tensor(1.2192, device='cuda:0', grad_fn=<SqrtBackward>)
Getting validation predictions...done! 0.05778025463223457..0.05778025463223457
Validation loss tensor(0.7400)
Validation Pearson 0.1565319490376306
Epoch #9

## Preprocessing

### English

1. Tokenise with spaCy language model
2. Remove stop words and punctuation
3. Normalise - lemmas

In [14]:
# Downloading spacy models for English

!spacy download en_core_web_md
!spacy link en_core_web_md en300 --force

     |████████████████████████████████| 95.4MB 814kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=a49473a29579e1d429d79548ab525630017c2af11f75936c97ddd80e22536f72
  Stored in directory: /tmp/pip-ephem-wheel-cache-arqmcjtz/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_md -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en300
You can now load the model via spacy.load('en300')


In [15]:
# Downloading stop words for English

from nltk import download
from nltk.corpus import stopwords

download('stopwords')
stop_words_en = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Get tokenizer

import spacy

nlp_en = spacy.load('en300')

In [0]:
def preprocess_en(sentence=None, *, keep_stopwords=False):
    def wrapper(sentence):
        text = sentence.lower()
        processed = [token.lemma_ for token in nlp_en.tokenizer(text)]
        processed = [token for token in processed if token.isalpha()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words_en]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

### Chinese

1. Tokenise with jieba
2. Remove stop words and punctuation

In [18]:
# Download stop words
FILE_STOP_WORDS_ZH = './chinese_stop_words.txt'

if not os.path.exists(FILE_STOP_WORDS_ZH):
    !wget -c https://github.com/Tony607/Chinese_sentiment_analysis/blob/master/data/chinese_stop_words.txt

with open(FILE_STOP_WORDS_ZH, 'r', encoding='utf-8') as f:
    stop_words_zh = [line.rstrip() for line in f]

--2020-02-16 19:58:45--  https://github.com/Tony607/Chinese_sentiment_analysis/blob/master/data/chinese_stop_words.txt
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘chinese_stop_words.txt’

chinese_stop_words.     [ <=>                ] 416.77K  --.-KB/s    in 0.03s   

2020-02-16 19:58:46 (13.5 MB/s) - ‘chinese_stop_words.txt’ saved [426769]



In [0]:
import jieba

def preprocess_zh(sentence=None, *, keep_stopwords=False):
    def wrapper(sentence):
        tokens = jieba.cut(sentence, cut_all=True)
        processed = [token for token in tokens if token.isalnum()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

## Language Vocabulary

In [0]:
class Language(object):

    SOS_TOKEN = '<SOS>'
    EOS_TOKEN = '<EOS>'
    UNK_TOKEN = '<UNK>'

    def __init__(self, name):
        self.name = name
        self.word2idx = {}
        self.word2count = {}
        self.idx2word = {0: self.SOS_TOKEN,
                         1: self.EOS_TOKEN,
                         2: self.UNK_TOKEN}
    
    def __len__(self):
        return len(self.idx2word)

    def add_sentence(self, sentence):
        for token in sentence:
            self.add_word(token)

    def add_word(self, word):
        if word not in self.word2idx:
            idx = len(self)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
        
        count = self.word2count.get(word, 0)
        self.word2count[word] = count + 1

    def sent_to_idxs(self, sent):
        return [self.word2idx.get(word, 2) for word in sent]
    
    def __repr__(self):
        return f'Language(name={self.name}) with {len(self)} words'

## Loading Data

In [21]:
# English data

preprocess_english = preprocess_en(keep_stopwords=True)

train_en_sents = [preprocess_english(sent) for sent in train_en]
val_en_sents = [preprocess_english(sent) for sent in val_en]
test_en_sents = [preprocess_english(sent) for sent in test_en]

EN = Language('EN')
for sent in train_en_sents:
    EN.add_sentence(sent)

print(EN)

print()
print('Sample sentence')
sample_sent_en = train_en_sents[42]
print(sample_sent_en)
print(EN.sent_to_idxs(sample_sent_en))

train_en_idxs = [EN.sent_to_idxs(sent) for sent in train_en_sents]
val_en_idxs = [EN.sent_to_idxs(sent) for sent in val_en_sents]
test_en_idxs = [EN.sent_to_idxs(sent) for sent in test_en_sents]

Language(name=EN) with 19248 words

Sample sentence
['all', 'of', 'the', 'artilleryman', 'record', 'a', 'wound', 'die']
[340, 31, 3, 341, 342, 59, 343, 344]


In [22]:
# Chinese data

preprocess_chinese = preprocess_zh(keep_stopwords=True)

train_zh_sents = [preprocess_chinese(sent) for sent in train_zh]
val_zh_sents = [preprocess_chinese(sent) for sent in val_zh]
test_zh_sents = [preprocess_chinese(sent) for sent in test_zh]

ZH = Language('ZH')
for sent in train_zh_sents:
    ZH.add_sentence(sent)

print(ZH)

print()
print('Sample sentence')
sample_sent_zh = train_zh_sents[42]
print(sample_sent_zh)
print(ZH.sent_to_idxs(sample_sent_zh))

train_zh_idxs = [ZH.sent_to_idxs(sent) for sent in train_zh_sents]
val_zh_idxs = [ZH.sent_to_idxs(sent) for sent in val_zh_sents]
test_zh_idxs = [ZH.sent_to_idxs(sent) for sent in test_zh_sents]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.774 seconds.
Prefix dict has been built successfully.


Language(name=ZH) with 23851 words

Sample sentence
['据', '记录', '所有', '6', '名', '炮兵', '都', '受伤', '了']
[483, 484, 485, 267, 486, 487, 488, 489, 17]


In [0]:
# Process scores

def prepare_score(score):
    return float(score)

train_scores = [prepare_score(score) for score in train_scores]
val_scores = [prepare_score(score) for score in val_scores]

In [0]:
# Datasets

train_en_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in train_en_idxs]
train_zh_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in train_zh_idxs]

train_pairs = list(zip(train_en_tensors, train_zh_tensors))
train_set = list(zip(train_pairs, train_scores))

val_en_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in val_en_idxs]
val_zh_tensors = [torch.LongTensor(sent_idxs) for sent_idxs in val_zh_idxs]

val_pairs = list(zip(val_en_tensors, val_zh_tensors))
val_set = list(zip(val_pairs, val_scores))

# val_pairs = list(zip(val_en_idxs, val_zh_idxs))
# test_pairs = list(zip(test_en_idxs, test_zh_idxs))

## Models

In [0]:
# Utilities

from scipy.stats.stats import pearsonr

def unzip(args):
    return zip(*args)

def RMSELoss(pred, target):
    return torch.sqrt(torch.mean((pred - target) ** 2))

### FFNN with trained embeddings

In [0]:
class FFNN(nn.Module):
    
    def __init__(self, *, en_vocab_size, zh_vocab_size, emb_dim):
        super().__init__()
        self.en_vocab_size = en_vocab_size
        self.zh_vocab_size = zh_vocab_size
        self.emb_dim = emb_dim

        self.en_embedding = nn.Embedding(self.en_vocab_size, self.emb_dim)
        self.zh_embedding = nn.Embedding(self.zh_vocab_size, self.emb_dim)

        self.en_hidden = nn.Linear(self.emb_dim, 1)
        self.zh_hidden = nn.Linear(self.emb_dim, 1)

        self.out = nn.Linear(2, 1)
    
    def forward(self, en_tensors, zh_tensors):
        en_emb = self.en_embedding(en_tensors)
        zh_emb = self.zh_embedding(zh_tensors)

        en_hid = F.relu(self.en_hidden(en_emb))
        zh_hid = F.relu(self.zh_hidden(en_emb))

        hid_concat = torch.stack((en_hid, zh_hid), axis=1).squeeze()
        score = self.out(hid_concat)
        return score.mean()

In [27]:
ffnn = FFNN(en_vocab_size=len(EN), zh_vocab_size=len(ZH), emb_dim=200)
ffnn.to(device)
print(ffnn)

ffnn_opt = torch.optim.Adam(ffnn.parameters(), lr=0.003)
loss_fn = RMSELoss

FFNN(
  (en_embedding): Embedding(19248, 200)
  (zh_embedding): Embedding(23851, 200)
  (en_hidden): Linear(in_features=200, out_features=1, bias=True)
  (zh_hidden): Linear(in_features=200, out_features=1, bias=True)
  (out): Linear(in_features=2, out_features=1, bias=True)
)


In [0]:
NUM_EPOCHS = 100

train_losses = []
val_losses = []
val_pearson = []

for eidx in range(NUM_EPOCHS):
    print(f'Epoch {eidx + 1}: \t', end=' ')
    ffnn.zero_grad()
    
    loss = 0
    for (en_tensor, zh_tensor), score in train_set:
        pred = ffnn(en_tensor.to(device), zh_tensor.to(device))
        loss += loss_fn(pred, score)

    loss /= len(train_set)
    train_losses.append(loss)
    
    print(f'train loss = {loss:.5f}\t', end='')

    # Validation loss
    val_loss = 0
    for (en_tensor, zh_tensor), score in val_set:
        pred = ffnn(en_tensor.to(device), zh_tensor.to(device))
        val_loss += loss_fn(pred, score)
    val_loss /= len(val_set)
    print(f'validation loss = {val_loss:.5f}\t', end='')
    val_losses.append(val_loss)

    # Validation score
    val_preds, val_targets = unzip([(ffnn(en_tensor.to(device), zh_tensor.to(device)).detach().cpu().numpy(), score)
                              for (en_tensor, zh_tensor), score in val_set])
    
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)

    pearson_score, _ = pearsonr(val_preds, val_targets)
    val_pearson.append(pearson_score)
    print(f'validation pearson = {pearson_score:.5f}\t')

    # Backpropagation
    loss.backward()
    ffnn_opt.step()

Epoch 1: 	 train loss = 0.87688	validation loss = 0.88592	validation pearson = 0.01748	
Epoch 2: 	 train loss = 0.86025	validation loss = 0.87088	validation pearson = 0.02019	
Epoch 3: 	 train loss = 0.84733	validation loss = 0.85943	validation pearson = 0.01939	
Epoch 4: 	 train loss = 0.83801	validation loss = 0.85154	validation pearson = 0.02470	
Epoch 5: 	 train loss = 0.82957	validation loss = 0.84433	validation pearson = 0.03254	
Epoch 6: 	 train loss = 0.82264	validation loss = 0.83841	validation pearson = 0.03838	
Epoch 7: 	 train loss = 0.81701	validation loss = 0.83367	validation pearson = 0.04050	
Epoch 8: 	 train loss = 0.81212	validation loss = 0.82953	validation pearson = 0.04263	
Epoch 9: 	 train loss = 0.80755	validation loss = 0.82555	validation pearson = 0.04476	
Epoch 10: 	 train loss = 0.80332	validation loss = 0.82175	validation pearson = 0.04683	
Epoch 11: 	 train loss = 0.79937	validation loss = 0.81817	validation pearson = 0.04816	
Epoch 12: 	 train loss = 0.795

### RNN Chain

In [0]:
class RNNChain(nn.Module):

    def __init__(self, *, en_vocab_size, zh_vocab_size, emb_dim):
        super().__init__()
        self.en_vocab_size = en_vocab_size
        self.zh_vocab_size = zh_vocab_size
        self.emb_dim = emb_dim

        self.en_embedding = nn.Embedding(self.en_vocab_size, self.emb_dim)
        self.zh_embedding = nn.Embedding(self.zh_vocab_size, self.emb_dim)

        self.en_rnn = nn.GRU(self.emb_dim, self.emb_dim, bidirectional=True)
        self.zh_rnn = nn.GRU(self.emb_dim, self.emb_dim, bidirectional=True)

        self.hidden = nn.Linear(self.emb_dim, 50)
        self.out = nn.Linear(50, 1)

    def forward(self, en_tensor, zh_tensor):
        en_emb = self.en_embedding(en_tensor)
        zh_emb = self.zh_embedding(zh_tensor)

        en_hidden = torch.zeros(2, 1, self.emb_dim, device=device)

        for word_idx in en_emb:
            word_idx = word_idx.view(1, 1, -1)
            _, en_hidden = self.en_rnn(word_idx, en_hidden)
    
        zh_hidden = en_hidden
        for word_idx in zh_emb:
            word_idx = word_idx.view(1, 1, -1)
            _, zh_hidden = self.zh_rnn(word_idx, zh_hidden)

        score = self.out(F.relu(self.hidden(zh_hidden[-1])))
        return score

In [0]:
USE_PREV = True

rnn = RNNChain(en_vocab_size=len(EN), zh_vocab_size=len(ZH), emb_dim=100)
rnn.to(device)

rnn_opt = torch.optim.Adam(rnn.parameters(), lr=0.003)
loss_fn = RMSELoss

NUM_EPOCHS = 100

state = {
    'curr_epoch': 1,
    'train_losses': [],
    'val_losses': [],
    'val_pearson': [],
}

if USE_PREV and os.path.exists(in_gdrive('rnn.pt')):
    print('Loading from Google Drive...', end=' ')
    rnn.load_state_dict(torch.load(in_gdrive('rnn.pt')))

    with open(in_gdrive('rnn.json'), 'r') as f:
        state = json.load(f)
    print('done!')


while state['curr_epoch'] <= NUM_EPOCHS:
    print(f'Epoch {state["curr_epoch"]}:')
    rnn.zero_grad()
    
    loss = 0
    print(f'Training {len(train_set)}: ', end='')
    for idx, ((en_tensor, zh_tensor), score) in enumerate(train_set):
        pred = rnn(en_tensor.to(device), zh_tensor.to(device)).squeeze()
        curr_loss = loss_fn(pred, score) 
        loss += curr_loss
        if idx % 500 == 0:
            print('.', end='')
    print()

    loss /= len(train_set)
    state['train_losses'].append(loss.detach().cpu().numpy().tolist())
    
    print(f'==>train loss = {loss:.5f}')

    # Validation loss
    val_loss = 0
    print(f'Validating loss {len(val_set)}: ', end='')
    for idx, ((en_tensor, zh_tensor), score) in enumerate(val_set):
        pred = rnn(en_tensor.to(device), zh_tensor.to(device))
        val_loss += loss_fn(pred, score)

        if idx % 100 == 0:
            print('.', end='')
    print()    

    val_loss /= len(val_set)
    print(f'==>validation loss = {val_loss:.5f}')
    state['val_losses'].append(val_loss.detach().cpu().numpy().tolist())

    # Validation score
    val_preds, val_targets = unzip([(rnn(en_tensor.to(device), zh_tensor.to(device)).squeeze().detach().cpu().numpy(), score)
                              for (en_tensor, zh_tensor), score in val_set])
    val_preds = np.array(val_preds)
    val_targets = np.array(val_targets)

    pearson_score, _ = pearsonr(val_preds, val_targets)
    state['val_pearson'].append(pearson_score)
    print(f'==>validation pearson = {pearson_score:.5f}')

    # Backpropagation
    print('Backpropagation...', end=' ')
    loss.backward()
    rnn_opt.step()
    print('done!')

    # Save
    print('Saving to Google Drive...', end=' ')
    torch.save(rnn.state_dict(), in_gdrive('rnn.pt'))

    state['curr_epoch'] += 1
    with open(in_gdrive('rnn.json'), 'w') as f:
        json.dump(state, f)

    print('done!\n')
    

In [0]:
class RNNChain(nn.Module):

    def __init__(self, *, vocab_size, emb_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim

        self.embedding = nn.Embedding(self.vocab_size, self.emb_dim)
        self.rnn = nn.GRU(self.emb_dim, self.emb_dim, bidirectional=True)

    def forward(self, x, hidden):
        emb = self.embedding(x)
        output = emb.view(1, 1, -1)
        output, hidden = self.rnn(output, hidden)

        return output, hidden

    def init_hidden(self):
        return torch.zeros(2, 1, self.emb_dim, device=device)

class RegressorLayer(nn.Module):

    def __init__(self, *, emb_dim):
        super().__init__()
        self.emb_dim = emb_dim

        self.hidden = nn.Linear(self.emb_dim, 50)
        self.out = nn.Linear(50, 1)

    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [0]:
en_model = RNNChain(vocab_size=len(EN), emb_dim=200)
zh_model = RNNChain(vocab_size=len(ZH), emb_dim=200)
regressor = RegressorLayer(emb_dim=200)

en_model.to(device)
zh_model.to(device)
regressor.to(device)

print(en_model)
print(zh_model)

LR = 0.003

en_opt = torch.optim.Adam(en_model.parameters(), lr=LR)
zh_opt = torch.optim.Adam(zh_model.parameters(), lr=LR)
regressor_opt = torch.optim.Adam(regressor.parameters(), lr=LR)

def RMSELoss(pred, target):
    return torch.sqrt(torch.mean((pred - target) ** 2))

loss_fn = RMSELoss

In [0]:
def train(en_tensor, zh_tensor, score):
    en_model.zero_grad()
    zh_model.zero_grad()

    en_hidden = en_model.init_hidden()

    for word_idx in en_tensor:
        hids, en_hidden = en_model(word_idx, en_hidden)
        print('Hids', hids.shape)
    
    # print('EN final hidden state', en_hidden)

    zh_hidden = en_hidden
    for word_idx in zh_tensor:
        _, zh_hidden = zh_model(word_idx, zh_hidden)
    
    # print('ZH final hidden state', zh_hidden)

    pred_score = regressor(zh_hidden).squeeze()

    loss = loss_fn(pred_score, score)
    
    # print('Loss', loss)    

    loss.backward()

    regressor_opt.step()
    zh_opt.step()
    en_opt.step()

    return loss.data

In [0]:

for eidx in range(100):
    loss = 0
    for (en_tensor, zh_tensor), score in train_set[:100]:
        loss += train(en_tensor.to(device), zh_tensor.to(device), score)
    loss /= 100
    print(loss)